In [20]:
import numpy as np
from bilby.core.result import read_in_result
from bilby.hyper.model import Model
from gwpopulation.models.redshift import PowerLawRedshift
from tqdm import tqdm

from gwpop.mass_models import matter_matters_pairing
from gwpop.spin_models import (
    iid_spin_magnitude_gaussian,
    iid_spin_orientation_gaussian_isotropic,
)
from pipe.gwpopulation_pipe_pdb import draw_true_values

## 1. CONFIGURATION

In [21]:
RESULT_FILE = "data/baseline5_widesigmachi2_mass_NotchFilterBinnedPairingMassDistribution_redshift_powerlaw_mag_iid_spin_magnitude_gaussian_tilt_iid_spin_orientation_result.hdf5"
N_SAMPLES = 1000  # int(1e6)
CHUNK_SIZE = int(1e3)
Z_MAX = 2.3


## 2. LOAD MAP PARAMETERS FROM BILBY RESULT FILE

In [22]:
# Load "Broken Power Law + 2 Peaks model" result
result = read_in_result(RESULT_FILE)
post = result.posterior.copy()

In [23]:
# --- MAP hyperparameters ---
# Extract the Maximum a Posteriori (MAP) parameters from a Bilby result.
# - Load the .hdf5 result file and copy the posterior samples.
# - Compute a score:
#     * If the prior is non-uniform, use log_likelihood + log_prior (true MAP).
#     * If the prior is uniform (constant, as in this case), log_prior adds nothing,
#       this means Maximum Likelihood (ML) and Maximum A Posteriori (MAP) coincide.
# - Select the sample that maximizes this score.

if "log_prior" in post and post["log_prior"].nunique() > 1:
    score = post.log_likelihood + post.log_prior
    max_likelihood_sample = post.iloc[np.argmax(score)]
else:
    max_likelihood_sample = post.iloc[np.argmax(post.log_likelihood)]

In [31]:
import pandas as pd

pd.set_option("display.max_rows", None)
max_likelihood_sample

A                           0.091462
A2                          0.828165
BHmax                     152.055979
BHmin                       7.763955
NSmax                       4.094744
NSmin                       1.176367
UPPERmax                   66.576705
UPPERmin                   38.277415
alpha_1                    -4.509283
alpha_2                    -0.902035
alpha_chi                  -0.013141
alpha_dip                  -1.679769
amax                        1.000000
beta_chi                   -0.942731
beta_pair_1                 0.964138
beta_pair_2                 2.160036
lamb                        2.406658
ln_bf_0                   -27.166847
ln_bf_1                   -38.760948
ln_bf_10                  -39.115001
ln_bf_100                 -42.231597
ln_bf_101                 -38.617247
ln_bf_102                 -38.718507
ln_bf_103                 -38.913520
ln_bf_104                 -44.246029
ln_bf_105                 -38.034746
ln_bf_106                 -38.446173
l

In [25]:
# Set minimum and maximum allowed masses for the model (can be tuned)
max_likelihood_sample["absolute_mmin"] = 0.5
max_likelihood_sample["absolute_mmax"] = 350.0

In [26]:
print("Max parameters:")
print(max_likelihood_sample["log_likelihood"])

Max parameters:
-4249.867040950891



## 3. CONSTRUCT POPULATION MODEL WITH MAP PARAMETERS

In [27]:
# Compose the model: mass, spin orientation, spin magnitude, redshift
model = Model(
    [
        matter_matters_pairing,
        iid_spin_orientation_gaussian_isotropic,
        iid_spin_magnitude_gaussian,
        PowerLawRedshift(z_max=Z_MAX),
    ],
    cache=False,
)

model

In [28]:
# Set the parameters from MAP sample
model.parameters.update(max_likelihood_sample)

## 4. DRAW SYNTHETIC POPULATION SAMPLES (IN CHUNKS)

In [29]:
N_SAMPLES = int(100)
n_chunks = int(np.ceil(N_SAMPLES / CHUNK_SIZE))

all_events = []
for i in tqdm(range(n_chunks), desc="Simulating events"):
    current_chunk_size = min(CHUNK_SIZE, N_SAMPLES - i * CHUNK_SIZE)
    events = draw_true_values(model=model, vt_model=None, n_samples=current_chunk_size)
    # all_events.append(events)

Simulating events:   0%|          | 0/1 [00:00<?, ?it/s]

model.prob(data).shape: (1000000,)


18:41 bilby INFO    : Sampling efficiency low. Total samples so far: 162430


model.prob(data).shape: (1000000,)


18:41 bilby INFO    : Sampling efficiency low. Total samples so far: 182321


...

18:41 bilby INFO    : Sampling efficiency low. Total samples so far: 455996
/Users/rk281962/Projects/CBC/GWTC-4-distribution/gwpop/mass_models.py:126: RuntimeWarning: overflow encountered in power
  notch_lower = 1.0 - A / ((1 + (NSmax / mass) ** n1) * (1 + (mass / BHmin) ** n2))


model.prob(data).shape: (1000000,)


18:41 bilby INFO    : Sampling efficiency low. Total samples so far: 547608


model.prob(data).shape: (1000000,)


Simulating events: 100%|██████████| 1/1 [00:08<00:00,  8.26s/it]


In [30]:
events

,mass_1,mass_ratio,a_1,a_2,cos_tilt_1,cos_tilt_2,redshift,mass_2
1133,1.484307,0.945063,0.308062,0.203302,0.507513,0.668238,0.602106,1.402764
7945,1.686981,0.967234,0.245483,0.097739,-0.934245,0.983071,0.939943,1.631705
10044,1.885763,0.705051,0.325317,0.390507,0.742669,0.595289,1.397566,1.329560
13143,1.266025,0.991539,0.050669,0.018240,0.914856,-0.374560,1.453190,1.255313
13361,9.728875,0.140123,0.473527,0.391022,0.708313,0.976816,1.226861,1.363238
20238,1.444287,0.965235,0.299611,0.266260,0.345779,0.918093,1.398243,1.394077
30853,1.318626,0.992420,0.317815,0.332953,-0.272564,0.426057,0.945790,1.308631
37534,1.823711,0.736303,0.308295,0.092261,0.582666,0.699596,0.491590,1.342805
38304,1.980880,0.592587,0.084560,0.102908,0.644859,0.862007,1.031226,1.173844
39207,3.434717,0.370235,0.130031,0.239320,0.619292,0.192250,1.337270,1.271653
